In [1]:
# Cell 1

import pylab as pl
import casadi as ca
import numpy as np
print(ca.__version__)

3.3.0


In [2]:
# Cell 2

pl.close("all")

# Parameters for experiments

## Duration and time points

hours = 24
controls_actions_per_hour = 4

t0 = 0.0;
tf = hours * 3600.0;
N = hours * controls_actions_per_hour;

time_points = pl.linspace(t0, tf, N + 1)

dt = (tf - t0) / N  # Duration of a time interval

## Number of storage layers in the HOT tank
nlayer_htes = 9

## The layer at which the HOT tank is connected to the load
layer_load_h = 6

## Number of storage layers in the COLD tank
nlayer_ctes = 0


In [3]:
# Cell 3

# Physical parameters of the system

## Component 1: CHP
Pmin_CHP_Th = 0 #kW_Th
Pmax_CHP_Th = 10
Pmin_CHP_El = 0
Pmax_CHP_El = 5 #kW_El
#Efficiencies 
Pel_eff_CHP = 0.3
Pth_eff_CHP = 0.59
# TODO: check the names
Pth_CHP_Nominal = 9.6e3 # in W
Pel_CHP_Nominal=1
CHP_eta_Thermal=1
CHP_eta_Electrical=1
CHP_Fuel_LHV=1
CHP_Fuel_HHV=12

## Component 2: HTES
Pmin_COIL_Th = 0
Pmax_COIL_Th = 5.8#kW_Th
Pth_eff_COIL = 0.95
#Storage Tank
HTES_Cap = 1.5 #m³
HTES_Warm = 75 + 273.15 #K Highest possible temperature in tank 
HTES_Cold = 35 + 273.15 #K Lowest possible temperature in tank
T_HTES_init_0 = 50 +273.15#Initial temperature in the Tank K
Q_HTES_init = 30 #kW Initial Power in Tank

# Dimensions of the tank
h=1
n=1
pi=3.14
rho=1
D=1.0
t=0.1

## Component 3: LOAD_H
Pth_LOAD_H = 1
T_LOAD_H_CC_FL = 1

## Component 4: HX3
## Heat exchangers
A_HX3 = 1
U_HX3 =1


## Component 5: HP
Pmin_HP_Th = 0
Pmax_HP_Th = 16#kW_Th
COP_HP = 4.45
T_HP_min = 10 #Temperature min to operate the HP.

v_dot_HP_HT=1
v_dot_HP_MT=1
m_dot_HP_HT_FL_Set=1
m_dot_HP_MT_FL_Set=1


## Component 6: OC2
v_dot_OC2=1
## Outdoor coils
A_OC =1
U_OC =1
RPM_max=1

## Component 7: AdCM
v_dot_AdCM_LT_set=1
v_dot_AdCM_MT_set=1
v_dot_AdCM_HT_set=1
SF = 0

## HX1?
A_HX12 = 1
A_HX1 = A_HX12
A_HX2 = A_HX12
A_HX3 = 1
U_HX12 = 1
U_HX1 = U_HX12 
U_HX2 = U_HX12
U_HX3 =1

## Component 8: OC1
v_dot_OC1=1

## Component 9: CTES

## Component 10: CCM
v_dot_CC_H = 1
T_CC_H_FL = 1
v_dot_CC_C =1
T_CC_C_FL =1

v_dot_CCM_MT=1
v_dot_CCM_LT=1
T_CCM_MT_RL=1
m_dot_CCM_MT_FL_Set=1
m_dot_CCM_LT_FL_Set=1

## Component 11: LOAD_C
Pth_LOAD_C = 1
T_LOAD_C_CC_FL = 1

## Component 12: OC3


# Constants

"Thermodynamical properties"
LHV_FUEL = 42600 #Kj/Kg
ro_FUEL = 853.5 #Kg/m³
rho_water = 977.8 #Kg/M3 (Cesar's name: ro_w)
Cp_w = 4.180 #KJ/Kg.K

#Fuel consumption of CHP in m³/h
vdot_fuel = 1.8/ro_FUEL

"Costs" 
#Electricity costs
# To add
#C_El_B= pl.array(epex_price_buy)
#C_El_S= pl.array(epex_price_sell)

#Fuel Costs
Fuel = 510 #eur/m³

#T_Amb is splitted into day and night temperatures
#T_Amb = 10 + 273.15 #K
T_A_N = 6 #Temperature at 0 in the night for the temp linspace
T_A_M = 15#Temperature at 16hrs in the evening for the temp linspace

T_HP_min = 10 #Temperature min to operate the HP.
T_HTES_init_0 = 50 +273.15#Initial temperature in the Tank K
Q_HTES_init = 30 #kW Initial Power in Tank


#Formulate the LP Problem
#Set the time Steps
#n= int(Time.size) + 1

#define the ambient temperature linear increase or decrease
T_Amb=[]
days = int(round((n-1)/24.0))


T_Amb_1 = pl.linspace(T_A_N, T_A_M, (16))
T_Amb_2 = pl.linspace(T_A_M, T_A_N, (24)-16)

T_amb = T_A_N

T_amb_K = 300 #arbitrary


lambda_eff = 1
kappa = 1

In [4]:
# Cell 4

# Secondary parameters to be used in equations (with conversion)
# All quantities here need to be derived from earlier notions
# No constant definition in this cell

m_dot_CC_H = v_dot_CC_H*rho_water/3600
m_dot_CC_C = v_dot_CC_C*rho_water/3600
m_dot_AdCM_LT_Set = v_dot_AdCM_LT_set*rho_water/3600
m_dot_AdCM_MT_Set = v_dot_AdCM_MT_set*rho_water/3600
m_dot_AdCM_HT_Set = v_dot_AdCM_HT_set*rho_water/3600

m_dot_HP_HT = v_dot_HP_HT*rho_water/3600
m_dot_HP_MT = v_dot_HP_MT*rho_water/3600

m_dot_OC1 = v_dot_OC1*rho_water/3600
m_dot_OC2 = v_dot_OC2*rho_water/3600

m_dot_CCM_MT = v_dot_CCM_MT*rho_water/3600
m_dot_CCM_LT = v_dot_CCM_LT*rho_water/3600

## Parameters for a layer in the HTES
zi = h / n;
mi = pi * (D / 2) ** 2 * zi * rho;
di = D - 2 * t;
Aamb = pi * D * zi;
Alayer = pi * di ** 2 / 4;


In [5]:
# Some equations for intermediate quantities that are not real variables
# Can be eliminated by hand

# Pth_AdCM_LT = f(T_HTES_t, T_AdCM_MT_RL, T_CTES_t)
# COP_AdCM = f(T_HTES_t, T_AdCM_MT_RL, T_CTES_t)
# Pth_CCM_LT = 
# Pth_AdCM_HT =
# Pth_AdCM_MT = 
# Pth_CCM_MT

In [6]:
# Cell 5

# Declaration of variables

## Number of control variables
nu = 4

## Number of state variables
nx = 1 + nlayer_htes + nlayer_ctes

## Number of algebraic states (depending on the way we define variables)
ny = 44

## States and controls
u = ca.SX.sym("u", nu) # Control
x = ca.SX.sym("x", nx ) # Differential states: temperatures in the tank
y = ca.SX.sym("x", ny) # Output variable

# Input variables
CHP_Switch  = u[0]
HP_Switch   = u[1]
AdCM_Switch = u[2]
CCM_Switch  = u[3]

CHP_ON_int  = u[0]
RevHP_HP_ON_int   = u[1]
AdCM_ON_int = u[2]
RevHP_CC_ON_int  = u[3]

# State variables
Pth_CHP_x = x[0]
T_HTES = [k for k in range(nlayer_htes)]
T_CTES = [k for k in range(nlayer_ctes)]
for k in range(nlayer_htes):
    T_HTES[k] = x[1+k]
for k in range(nlayer_ctes):
    T_CTES[k] = x[1+nlayer_htes+k]

# Add names to conform to OpenModelica variables
T_HTES_t = T_HTES[nlayer_htes-1]  # top layer
T_HTES_l = T_HTES[layer_load_h-1] # layer output to the hot load
T_HTES_b = T_HTES[0] # bottom layer

# Output variables. We will re-order the component y[i] from 0, 1, ...
k = 0  # to increase this index when defining variables' names
mdot_FUEL    = y[k]

## Component 1: CHP
k = k+1
Pth_CHP      = y[k]
k = k+1
Pel_CHP      = y[k]
k = k+1
mdot_CHP_nonzero = y[k]
k = k+1
m_dot_CHP_set = y[k]
k = k+1
mdot_CHP     = y[k]
k = k+1
T_CHP_FL     = y[k]

## Component 2: HTES - now these are all state variables
#T_HTES_t     = y[7]
#T_HTES_l     = y[8]
#T_HTES_b     = y[9]

## Component 3: LOAD_H
k = k+1
Pth_LOAD_H = y[k]
k = k+1
mdot_LOAD_H  = y[k]
k = k+1
T_LOAD_H_FL  = y[k]
k = k+1
T_LOAD_H_CC_RL = y[k]

## Component 4: HX3
k = k+1
T_HX3_FL     = y[k]

## Component 5: HP
k = k+1
Pel_HP = y[k]
k = k+1
Pth_HP_HT = y[k]
k = k+1
Pth_HP_MT = y[k]
k = k+1
T_HP_HT_FL   = y[k]
k = k+1
T_HP_HT_RL   = y[k]
k = k+1
T_HP_MT_FL   = y[k]
k = k+1
T_HP_MT_RL   = y[k]

## Component 6: OC2
k = k+1
Pel_OC2 = y[k]
k = k+1
v_dot_OC2 = y[k]
k = k+1
T_OC2_FL  = y[k]
k = k+1
T_OC2_RL  = y[k]

## Component 7: AdCM
k = k+1
T_AdCM_HT_FL = y[k]
k = k+1
T_AdCM_MT_FL = y[k]
k = k+1
T_AdCM_MT_RL = y[k]
k = k+1
T_AdCM_LT_FL = y[k]
Pel_AdCM = y[k]

## Component 8: OC1
k = k+1
Pel_OC1 = y[k]
k = k+1
v_dot_OC1 = y[k]
k = k+1
T_OC1_FL     = y[k]
k = k+1
T_OC1_RL     = y[k]

## Component 9: CTES
k = k+1
T_CTES_t     = y[k]
k = k+1
T_CTES_b     = y[k]

## Component 10: CCM
k = k+1
Pel_CCM = y[k]
k = k+1
Pth_CCM_LT = y[k]
k = k+1
Pth_CCM_MT = y[k]
k = k+1
T_CCM_LT_FL  = y[k]
k = k+1
T_CCM_MT_FL  = y[k]
k = k+1
T_LOAD_C_CC_RL = y[k]

## Component 11: LOAD_C
k = k+1
Pth_LOAD_C = y[k]
k = k+1
mdot_LOAD_C = y[k]
k = k+1
T_LOAD_C_FL = y[k]

## Component 12: OC3
k = k+1
Pel_OC3 = y[k]








#to_be_checked_and_added


In [7]:
# Cell 6

# Secondary variables
v_dot_LOAD_H = mdot_LOAD_H / rho_water
v_dot_LOAD_C = mdot_LOAD_C / rho_water
v_dot_FUEL = mdot_FUEL / rho_water
v_dot_CHP = mdot_CHP / rho_water

In [8]:
# Cell 7

# Initial values of variable at time t=0

## Initial states
Pth_CHP_x_t0 = 1

T_HTES_layer_top_t0 = 35.0
T_HTES_layer_bottom_t0 = 25.0
T_HTES_t0 = pl.linspace(T_HTES_layer_top_t0, T_HTES_layer_bottom_t0, nlayer_htes)

T_CTES_layer_top_t0 = 25.0
T_CTES_layer_bottom_t0 = 10.0
T_CTES_t0 = pl.linspace(T_CTES_layer_top_t0, T_CTES_layer_bottom_t0, nlayer_ctes)

x_0 = [Pth_CHP_x_t0] + list(T_HTES_t0) + list(T_CTES_t0)


## Initial controls

chp_status_init = 0.0 * pl.ones(time_points.size - 1)

u_init = chp_status_init # to check

u_0 = [1, 1, 0, 0]




In [16]:
# Cell 8

# Equations
# TODO: make the separate lists of equations from Open Modelica initiated model,
#       one file for differential equations, one file for algebraic equations

## Differential equations: xdot = f(x,z,u)
dxdt = []
## Use equations like:
# dxdt.append((1.0 / m_s_i) * status_CHP * (mdot_CHP_to_9 * T_out_CHP - mdot_9_to_8 * T_s[8])) # T_9_dot
# ...
# dxdt = ca.vertcat(*dxdt)

# For the power of the CHP, the first state:
# d(Pth_CHP_x)/dt * 560.794 + Pth_CHP_x = Pth_CHP_Nominal * CHP_Switch
dxdt.append((1/560.794) * Pth_CHP_Nominal * CHP_Switch - Pth_CHP_x)

# Top layer of HTES is indexed nlayer_htes-1, note that the first index is 0
dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[nlayer_htes-2] - T_HTES[nlayer_htes-1]) / zi + 4.18 * (mdot_CHP * (T_CHP_FL - T_HTES[nlayer_htes-2])) -20.0 * Aamb * kappa * (T_HTES[nlayer_htes-1] - T_amb_K) ) )
# Loop from the layer next-to-top to layer_load_h
for k in range(nlayer_htes-2, layer_load_h-1, -1):
    dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi + 4.18 * (mdot_CHP * (T_HTES[k+1] - T_HTES[k])) -20.0 * Aamb * kappa * (T_HTES[k] - T_amb_K) ) )

# To deal with the possibility that there could be reverse flow from bottom to the layer_load_h, if mdot_LOAD_H > mdot_CHP:
# Make use of this formula: (x-y)*(a+b)/2 + abs(x-y)*(a-b)/2 = 
#  = (x-y)*a, if x>=y (positive)
#  = (x-y)*b, if x<y (negative)
# Note for Casadi: basic Python functions like abs, max, min are replaced by fabs, fmax, fmin 
# https://groups.google.com/d/msgid/casadi-users/d441bbf3-888d-4cfe-848b-2c2b9efb29f6%40googlegroups.com?utm_medium=email&utm_source=footer

# At the layer that water coming to the load, with index in the Python list = layer_load_h-1, use the formula above,
#   with: x=mdot_LOAD_H, y=mdot_CHP, a=T_HTES[layer_load_h-2], b=T_HTES[layer_load_h-1]
# then we can represent the heat that the layer RECEIVES from the liquid coming from the layer below (+ or -):
#   (mdot_LOAD_H-mdot_CHP)*T_HTES[layer_load_h-2] > 0, if mdot_LOAD_H>mdot_CHP, i.e. receiving water flowing up
#   (mdot_LOAD_H-mdot_CHP)*T_HTES[layer_load_h-1] < 0, if mdot_LOAD_H<mdot_CHP, i.e. losing water flowing down
dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[layer_load_h] - 2*T_HTES[layer_load_h-1] + T_HTES[layer_load_h-2]) / zi + 4.18 * (mdot_CHP * T_HTES[layer_load_h] - mdot_LOAD_H * T_LOAD_H_FL + (mdot_LOAD_H - mdot_CHP)*(T_HTES[layer_load_h-2] + T_HTES[layer_load_h-1])/2 + ca.fabs(mdot_LOAD_H - mdot_CHP)*(T_HTES[layer_load_h-2] - T_HTES[layer_load_h-1])/2  ) - 20.0 * Aamb * kappa * (T_HTES[layer_load_h-1] - T_amb_K) ) )
# for dT_HTES[layer_load_h-1]/dt

# Loop from the layer below the layer having output to load (index = layer_load_h-2), to the 2nd layer (index = 1):
# Use the formula above, with: x=mdot_LOAD_H, y=mdot_CHP, a=T_HTES[k-1]-T_HTES[k], b=T_HTES[k]-T_HTES[k+1]
# then the heat that the layer receives by moving liquid is:
#   (mdot_LOAD_H-mdot_CHP)*(T_HTES[k-1]-T_HTES[k]), if mdot_LOAD_H>mdot_CHP, i.e. water flowing upward
#   (mdot_LOAD_H-mdot_CHP)*(T_HTES[k]-T_HTES[k+1]), if mdot_LOAD_H<mdot_CHP, i.e. water flowing downward
for k in range(layer_load_h-2, 0, -1):
    dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[k+1] - 2*T_HTES[k] + T_HTES[k-1]) / zi + 4.18 * ( (mdot_LOAD_H - mdot_CHP)*(T_HTES[k-1] - T_HTES[k+1])/2 + ca.fabs(mdot_LOAD_H - mdot_CHP)*(T_HTES[k-1] - 2*T_HTES[k] + T_HTES[k+1])/2 ) - 20.0 * Aamb * kappa * (T_HTES[k] - T_amb_K) ) )

# The bottom layer, which corresponds to T_HTES[0]:
# the heat that the layer RECEIVES from the liquid coming from the 2nd layer:
#   (mdot_CHP-mdot_LOAD_H)*T_HTES[1] > 0, if mdot_CHP>mdot_LOAD_H, i.e. receiving water flowing down
#   (mdot_CHP-mdot_LOAD_H)*T_HTES[0] < 0, if mdot_CHP<mdot_LOAD_H, i.e. losing water flowing up

dxdt.append(1/(4.18 * mi) * ( Alayer * lambda_eff * (T_HTES[1] - T_HTES[0]) / zi + 4.18 * ( mdot_LOAD_H*T_LOAD_H_CC_RL - mdot_CHP*T_HTES[0] + (mdot_CHP - mdot_LOAD_H)*(T_HTES[1] - T_HTES[0])/2 + ca.fabs(mdot_CHP - mdot_LOAD_H)*(T_HTES[1] - T_HTES[0])/2 ) - 20.0 * Aamb * kappa * (T_HTES[0] - T_amb_K) ) )

dxdt = ca.vertcat(*dxdt)
#

## Algebraic equation, left hand side = 0
f_z = []
## Use equations like:
# f_z.append(P_th_CC/c_p - mdot_6_to_LOAD * (T_s[5] - T_LOAD_RL))
# f_z = ca.vertcat(*f_z)

f_z.append(- v_dot_CHP + 0.433352645 + -0.01514531 * T_HTES_b + 0.00024329 * T_HTES_b ** 2.0)
f_z.append(- T_CHP_FL + (T_HTES_b+273.15) + 0.2392344497607656 * Pth_CHP / m_dot_CHP_set)
f_z.append(- v_dot_FUEL + 1000.0 * (Pel_CHP + Pth_CHP) / (853.5 * (CHP_eta_Thermal + CHP_eta_Electrical) * CHP_Fuel_HHV) )
f_z.append(- Pth_LOAD_H + 4.18 * m_dot_CC_H * (T_LOAD_H_CC_FL - T_LOAD_H_CC_RL) )
f_z.append(- Pth_LOAD_H + 4.18 * (v_dot_LOAD_H*rho_water/3600) * ((T_LOAD_H_FL+273.15) - (T_LOAD_H_CC_RL+273.15)) )
#f_z.append(- COP + -0.049623287373 + 0.01893348591 * T_CTES_t + 0.013340776694 * T_AdCM_HT_FL + 0.017822939671 * T_AdCM_MT_RL + -0.001280352166 * T_CTES_t ** 2.0 + -0.000190832894 * T_AdCM_HT_FL ** 2.0 + -0.001993352016 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (-0.000334095159 * T_AdCM_HT_FL + 0.001455689548 * T_AdCM_MT_RL) + 0.000569253554 * T_AdCM_HT_FL * T_AdCM_MT_RL + 1.3421174e-05 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL)
#f_z.append(- Pth_AdCM_LT + AdCM_ON_int * (4.07950934099 + 0.04152472361 * T_CTES_t + 0.160630808297 * T_AdCM_HT_FL + -0.859860168466 * T_AdCM_MT_RL + 0.003462744142 * T_CTES_t ** 2.0 + -0.001049096999 * T_AdCM_HT_FL ** 2.0 + 0.015142231276 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (0.016955368833 * T_AdCM_HT_FL + -0.016151596215 * T_AdCM_MT_RL) + -0.001917799045 * T_AdCM_HT_FL * T_AdCM_MT_RL + -0.000200778961 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL))
#f_z.append(- Pth_AdCM_HT + Pth_AdCM_LT / COP - SF )
f_z.append(- Pth_HP_HT + HP_Switch * (9.0 + 0.294510922 * T_HP_MT_FL + T_HP_HT_RL * (0.064700246 + 0.002953381 * T_HP_MT_FL) + -0.001625553 * T_HP_MT_FL ** 2.0 + -0.001627312 * T_HP_HT_RL ** 2.0) )
f_z.append(- Pth_HP_MT + Pth_HP_HT - Pel_HP )
f_z.append(- (T_HP_HT_FL+273.15) + (T_HP_HT_RL+273.15) + 0.2392344497607656 * Pth_HP_HT / m_dot_HP_HT_FL_Set)
f_z.append(- (T_HP_MT_FL+273.15) + (T_HP_MT_FL+273.15) + -0.2392344497607656 * Pth_HP_MT / m_dot_HP_MT_FL_Set )
f_z.append(- Pel_HP + HP_Switch * (1.733202228 + -0.007333788 * T_HP_MT_FL + T_HP_HT_RL * (0.019283658 + 0.000450498 * T_HP_MT_FL) + -8.304799999999999e-05 * T_HP_MT_FL ** 2.0 + 0.000671146 * T_HP_HT_RL ** 2.0) )

f_z = ca.vertcat(*f_z)


In [17]:
# For code testing
#for k in range(nlayer_htes-2, layer_load_h, -1):
#    print(k)

dxdt.shape

(10, 1)

In [18]:
# TODO: bring new equations into the form of an optimal control problem
#### (below is untouched)

y
T_CTES2 = []

T_CTES2.append(x[5])
T_CTES2.append(x[6])

x
x[6]=10
T_CTES2

[SX(x_5), SX(10)]

In [19]:
x

SX([x_0, x_1, x_2, x_3, x_4, x_5, 10, x_7, x_8, x_9])

In [20]:
# Cell 9

# Constraints

T_min_all = 0.0
T_max_all = 100.0

x_lbw = list(T_min_all*pl.ones(nlayer_htes))
x_ubw = list(T_max_all*pl.ones(nlayer_htes))

#zeros_z = list(0.0*pl.ones(f_z.numel()))
#zeros_x = list(0.0*pl.ones(dxdt.numel()))

In [21]:
# Cell 10

# Create dynamical model in form of a DAE (Differential Algebraic Equations) system

dae = {'x':x, 'z':y, 'p':u, 'ode':dxdt, 'alg':f_z}
#opts = {'tf':0.5} # interval length
I = ca.integrator('I','collocation', dae)

f = ca.Function('f', [x, y, u], [dxdt,f_z], ['x', 'z', 'p'], ['ode','alg'])
fz = ca.Function('fz', [y, x, u], [f_z], ['z','x', 'p'], ['alg'])
G1 = ca.rootfinder('G1','newton',fz)

# Test simulation
z_0_guess = [0.511 for i in range(ny)]
outz=G1(z=z_0_guess, x=x_0, p=u_0)
z_0_solve=outz['alg']

z_start=z_0_solve

r = I(x0=x_0,z0=z_start,p=u_0)
print(x_0)
print(r['xf'])

print(z_start)
print(r['zf'])

# <codecell>

# Optimal control problem formulation
X = [ ca.MX.sym("x", N_s) for i in range(N+1)]
U = [ ca.MX.sym("u", 2) for i in range(N)]
# X and U are lists of MX variables


RuntimeError: .../casadi/core/integrator.cpp:1378: Assertion "de_in[DE_Z].size()==de_out[DE_ALG].size()" failed:
Dimension mismatch for 'alg'

In [46]:
## Equations copied from Open Modelica initiated model, with variable names changed to Casadi name set
#560.794 * der(Pth_CHP) + Pth_CHP = Pth_CHP * CHP_ON_int 
v_dot_CHP = 0.433352645 + -0.01514531 * T_HTES_b + 0.00024329 * T_HTES_b ** 2.0
CHP_H_W_T_M_FL_K = (T_HTES_b+273.15) + 0.2392344497607656 * Pth_CHP / m_dot_CHP_set
v_dot_FUEL = 1000.0 * (Pel_CHP + Pth_CHP) / (853.5 * (CHP_eta_Thermal + CHP_eta_Electrical) * CHP_Fuel_HHV) 
Pth_CC = 4.18 * m_dot_CC_H * (T_LOAD_H_CC_FL - T_LOAD_H_CC_RL)
Pth_CC = 4.18 * (v_dot_LOAD_H*rho_water/3600) * ((T_LOAD_H_FL+273.15) - (T_LOAD_H_CC_RL+273.15))
COP = -0.049623287373 + 0.01893348591 * T_CTES_t + 0.013340776694 * T_AdCM_HT_FL + 0.017822939671 * T_AdCM_MT_RL + -0.001280352166 * T_CTES_t ** 2.0 + -0.000190832894 * T_AdCM_HT_FL ** 2.0 + -0.001993352016 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (-0.000334095159 * T_AdCM_HT_FL + 0.001455689548 * T_AdCM_MT_RL) + 0.000569253554 * T_AdCM_HT_FL * T_AdCM_MT_RL + 1.3421174e-05 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL
Pth_AdCM_LT = AdCM_ON_int * (4.07950934099 + 0.04152472361 * T_CTES_t + 0.160630808297 * T_AdCM_HT_FL + -0.859860168466 * T_AdCM_MT_RL + 0.003462744142 * T_CTES_t ** 2.0 + -0.001049096999 * T_AdCM_HT_FL ** 2.0 + 0.015142231276 * T_AdCM_MT_RL ** 2.0 + T_CTES_t * (0.016955368833 * T_AdCM_HT_FL + -0.016151596215 * T_AdCM_MT_RL) + -0.001917799045 * T_AdCM_HT_FL * T_AdCM_MT_RL + -0.000200778961 * T_CTES_t * T_AdCM_HT_FL * T_AdCM_MT_RL)
Pth_AdCM_HT = Pth_AdCM_LT / COP - SF 
Pth_AdCM_MT = Pth_AdCM_HT + Pth_AdCM_LT 
T_AdCM_LT_FL_K = (T_CTES_t+273.15) + -0.2392344497607656 * Pth_AdCM_LT / m_dot_AdCM_LT_Set
T_AdCM_MT_FL_K = (T_AdCM_MT_RL+273.15) + 0.2392344497607656 * Pth_AdCM_MT / m_dot_AdCM_MT_Set
ADCM_C_W_T_M_HT_FL_K = (T_AdCM_HT_FL+273.15) + -0.2392344497607656 * Pth_AdCM_HT / m_dot_AdCM_HT_Set
Pth_HP_HT = HP_Switch * (9.0 + 0.294510922 * T_HP_MT_FL + T_HP_HT_RL * (0.064700246 + 0.002953381 * T_HP_MT_FL) + -0.001625553 * T_HP_MT_FL ** 2.0 + -0.001627312 * T_HP_HT_RL ** 2.0)
Pth_HP_MT = Pth_HP_HT - Pel_HP 
T_HP_HT_FL__K = (T_HP_HT_RL+273.15) + 0.2392344497607656 * Pth_HP_HT / m_dot_HP_HT_FL_Set
RevHP_HC_W_T_M_LT_FL__K = (T_HP_MT_FL+273.15) + -0.2392344497607656 * Pth_HP_MT / m_dot_HP_MT_FL_Set
Pel_HP = HP_Switch * (1.733202228 + -0.007333788 * T_HP_MT_FL + T_HP_HT_RL * (0.019283658 + 0.000450498 * T_HP_MT_FL) + -8.304799999999999e-05 * T_HP_MT_FL ** 2.0 + 0.000671146 * T_HP_HT_RL ** 2.0)
Pth_CCM_LT = CCM_Switch * (9.0 + 0.308329176 * T_CCM_LT_FL + 0.045285097 * T_CCM_MT_RL + 0.002252906 * T_CCM_LT_FL * T_CCM_MT_RL + -0.001213212 * T_CCM_LT_FL ** 2.0 + -0.002264659 * T_CCM_MT_RL ** 2.0)
Pel_CCM = CCM_Switch * (1.833202228 + -0.007333788 * T_CCM_LT_FL + 0.019283658 * T_CCM_MT_RL + 0.000450498 * T_CCM_LT_FL * T_CCM_MT_RL + -8.304799999999999e-05 * T_CCM_LT_FL ** 2.0 + 0.000671146 * T_CCM_MT_RL ** 2.0)
Pth_CCM_MT = Pel_CCM + Pth_CCM_LT
RevHP_HC_W_T_M_LT_FL_K = (T_CCM_LT_FL+273.15) + -0.2392344497607656 * Pth_CCM_LT / m_dot_CCM_LT_FL_Set
T_CCM_MT_FL_K = (T_CCM_MT_RL+273.15) + 0.2392344497607656 * Pth_CCM_MT / m_dot_CCM_MT_FL_Set
Pth_CC = 4.18 * m_dot_CC_C * (T_LOAD_C_CC_RL - T_LOAD_C_CC_FL) 
Pth_CC = 4.18 * (v_dot_LOAD_C*rho_water/3600) * ((T_LOAD_C_CC_RL+273.15) - (T_LOAD_C_FL+273.15)) 
#4.18 * mi * der(HTES_H_W_T_M_IT_K[1]) = Alayer * lambda_eff * (HTES_H_W_T_M_IT_K[2] - HTES_H_W_T_M_IT_K[1]) / zi + 4.18 * (m_dot_LOAD * (T_HTES_LOAD_RL_K - HTES_H_W_T_M_IT_K[1]) + m_dot_AdCM_HT * (T_HTES_AdCM_RL_K - HTES_H_W_T_M_IT_K[1]) + d_pos * m_dot[2] * (HTES_H_W_T_M_IT_K[2] - HTES_H_W_T_M_IT_K[1])) + -20.0 * Aamb * kappa * (HTES_H_W_T_M_IT_K[1] - T_amb_K)
#4.18 * mi * der(HTES_H_W_T_M_IT_K[2]) = Alayer * lambda_eff * (HTES_H_W_T_M_IT_K[1] + -2.0 * HTES_H_W_T_M_IT_K[2] + HTES_H_W_T_M_IT_K[3]) / zi + 4.18 * m_dot[2] * (d_pos * (HTES_H_W_T_M_IT_K[3] - HTES_H_W_T_M_IT_K[2]) + d_neg * (HTES_H_W_T_M_IT_K[2] - HTES_H_W_T_M_IT_K[1])) + COIL_H_E_PT_M / /*Real*/(n) - Aamb * kappa * (HTES_H_W_T_M_IT_K[2] - T_amb_K)
#Chot = 3.66736 * (v_dot_OC1*rho_water/3600)
#Ccold = 1.005 * (v_dot_air_OC1*rho_water/3600)
#Cmin = min(Chot, Ccold)
#Cmax = max(Chot, Ccold)
#qmax = Cmin * (T_OC1_RL - T)
#q = eff * qmax
#eff = (1.0 - exp(NTU * (Cr - 1.0))) / (1.0 - Cr * exp(NTU * (Cr - 1.0)))
#RPM_max / RPM_real = Volt_max / Volt_real
#Pel_max / Pel_OC1 = (RPM_max / RPM_real) ** 3.0
#v_dot_air_max / v_dot_air_real = RPM_max / RPM_real
#Chot = 3.66736 * (v_dot_OC2*rho_water/3600)
#Ccold = 1.005 * (v_dot_air_OC2*rho_water/3600)
#Cmin = min(Chot, Ccold)
#Cmax = max(Chot, Ccold)
#qmax = Cmin * (T_OC2_RL - T)
#q = eff * qmax
#eff = (1.0 - exp(NTU * (Cr - 1.0))) / (1.0 - Cr * exp(NTU * (Cr - 1.0)))
#RPM_max / RPM_real = Volt_max / Volt_real
#Pel_max / Pel_OC2 = (RPM_max / RPM_real) ** 3.0
#v_dot_air_max / v_dot_air_real = RPM_max / RPM_real
#Chot = 4.18 * m_dot_OC
#Ccold = 1.005 * m_dot_air
#Cmin = min(Chot, Ccold)
#Cmax = max(Chot, Ccold)
#qmax = Cmin * (T_CCM_MT_RL - T)
#q = eff * qmax
#eff = (1.0 - exp(NTU * (Cr - 1.0))) / (1.0 - Cr * exp(NTU * (Cr - 1.0)))
#q = Chot * (T_CCM_MT_RL - T_CCM_MT_FL)
#q = Ccold * (T_air_out - T)
#RPM_max / RPM_real = Volt_max / Volt_real
#Pel_max / Pel_OC3 = (RPM_max / RPM_real) ** 3.0
#v_dot_air_max / v_dot_air_real = RPM_max / RPM_real
#4.18 * mi * der(CTES_H_W_T_M_IT_K[1]) = 4.18 * (m_dot_AdCM * (T_CTES_AdCM_In_K - CTES_H_W_T_M_IT_K[1]) + m_dot_RevHP * (T_CTES_RevHP_In_K - CTES_H_W_T_M_IT_K[1])) + Alayer * lambda_eff * (CTES_H_W_T_M_IT_K[2] - CTES_H_W_T_M_IT_K[1]) / zi + 4.18 * d_neg * m_dot[2] * (CTES_H_W_T_M_IT_K[1] - CTES_H_W_T_M_IT_K[2]) - Aamb * kappa * (CTES_H_W_T_M_IT_K[1] - T_amb_K)
#4.18 * mi * der(CTES_H_W_T_M_IT_K[2]) = Alayer * lambda_eff * (CTES_H_W_T_M_IT_K[1] + -2.0 * CTES_H_W_T_M_IT_K[2] + CTES_H_W_T_M_IT_K[3]) / zi + 4.18 * m_dot[2] * (d_neg * (CTES_H_W_T_M_IT_K[2] - CTES_H_W_T_M_IT_K[3]) + d_pos * (CTES_H_W_T_M_IT_K[1] - CTES_H_W_T_M_IT_K[2])) - Aamb * kappa * (CTES_H_W_T_M_IT_K[2] - T_amb_K)
